In [0]:
from pyspark.sql import  SparkSession
import requests
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DateType,DoubleType
from pyspark.sql.functions import to_date


api_url = "https://script.google.com/macros/s/AKfycbx7A9tr8uHmTNGYv5Q2O24cthTYUzZPd1uqaFvHsBNShDmduvYZazpvgy3bY3-agt5e0g/exec"

response = requests.get(url=api_url)
if response.status_code == 200:
    result = response.json()

else:
    print("API request failed with status code: ",response.status_code)

data=[i[0].split(",") for i in result['rows']]
data = [[int(row[0]), *row[1:]] for row in data]


schema=StructType([
    StructField('product_id',IntegerType(),True),
    StructField('customer_id',StringType(),True),
    StructField('order_date',StringType(),True),
    StructField('location',StringType(),True),
    StructField('source_order',StringType(),True)

])

df_api = spark.createDataFrame(data,schema=schema)
df_api=df_api.withColumn('order_date',to_date(df_api['order_date'],'yyyy-MM-dd'))

schema=StructType([
    StructField('product_id',IntegerType(),True),
    StructField('customer_id',StringType(),True),
    StructField('order_date',DateType(),True),
    StructField('location',StringType(),True),
    StructField('source_order',StringType(),True)

])
df_file= spark.read.csv("/FileStore/tables/sales_csv.txt",schema=schema)

schema_menu=StructType([
    StructField('product_id',IntegerType(),True),
    StructField('product_name',StringType(),True),
    StructField('price',DoubleType(),True)
])
df_menu=spark.read.csv("/FileStore/tables/menu_csv.txt",
                       schema=schema_menu)

df=df_api.union(df_file)

# Transformation
from pyspark.sql.functions import month, year, quarter
df=df.withColumn('order_year',year('order_date'))
df=df.withColumn('order_quarter',quarter('order_date'))
df=df.withColumn('order_month',month('order_date'))

df_final=df.join(df_menu,on='product_id',how='inner')

#load
output_path='/FileStore/tables/output'
df_final.coalesce(1).write.mode("overwrite").parquet(output_path)